In [1]:
import pandas as pd
import numpy as np
from ServiceLayerRecommender import RecommenderLayer
import Constants
import ServiceLayerDao
import Constants

2023-05-11 06:53:43.182244: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-11 06:53:43.184954: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-11 06:53:43.226657: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-11 06:53:43.227500: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-11 06:53:43.999530: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [5]:
df_ex = pd.read_csv("./resources/data/exercise.csv", header=0)
df_workout = pd.read_csv("./resources/data/workout.csv", header=0)
df_user = pd.read_csv("./resources/data/user.csv", header=0)
df_derived_ex = pd.read_csv("./resources/data/exercise_derived.csv", header=0)

In [84]:
# print(df_ex.head(1))
# print(df_ex.dtypes)
# print(df_workout.head(1))
# print(df_workout.dtypes)
# print(df_user.head(1))
# print(df_user.dtypes)
# print(df_derived_ex.head(1))
# print(df_derived_ex.dtypes)

In [85]:
the_columns_exd = df_derived_ex.columns
the_columns_ex = df_ex.columns

In [98]:
print(the_columns_exd)
print(the_columns_ex)

Index(['id', 'name', 'exercise_importance', 'exercise_cost',
       'exercise_recurrance_factor_per_cycle', 'exercise_rpe',
       'exercise_overall_fun_factor', 'exercise_exhaustion_factor',
       'missed_in_cycle_count'],
      dtype='object')
Index(['id', 'name', 'exercise_type', 'muscles_targeted', 'calories_10_min',
       'fun_factor', 'is_compound', 'how_critical'],
      dtype='object')


In [137]:
# Validation the merged columns will be 15
k = set(df_ex.columns)
c = set(df_derived_ex.columns)
for i in c:
    k.add(i)
print(len(k))
k

15


{'calories_10_min',
 'exercise_cost',
 'exercise_exhaustion_factor',
 'exercise_importance',
 'exercise_overall_fun_factor',
 'exercise_recurrance_factor_per_cycle',
 'exercise_rpe',
 'exercise_type',
 'fun_factor',
 'how_critical',
 'id',
 'is_compound',
 'missed_in_cycle_count',
 'muscles_targeted',
 'name'}

In [118]:
# If bicep is in user focus column
"bicep" in df_user[df_user['name'] == 'Mike']['focused_muscle_group'].item().split()

True

In [170]:
def update_weights(row):
    if "bicep" in df_user[df_user['name'] == 'Mike']['focused_muscle_group'].item().split():
        # Update all rows who have bicep in muscle_targeted
        if 'bicep' in row['muscles_targeted'].split():
            id = row['id']
            old_val = df_derived_ex[df_derived_ex['id'] == id]['exercise_importance'].item()
            df_derived_ex.loc[df_derived_ex['id'] == id, 'exercise_importance'] = old_val + 0.37

In [2]:
# Rule trial for mapper
# Poc for mapping iterator
def init():
    df_ex = pd.read_csv("./resources/data/exercise.csv", header=0)
    df_workout = pd.read_csv("./resources/data/workout.csv", header=0)
    df_user = pd.read_csv("./resources/data/user.csv", header=0)
    df_derived_ex = pd.read_csv("./resources/data/exercise_derived.csv", header=0)

    df_derived_ex["exercise_importance"] = 1
    df_derived_ex["exercise_cost"] = 0
    # print(df_derived_ex.head(10))
    return df_ex, df_workout, df_user, df_derived_ex

In [3]:
df_ex, df_workout, df_user, df_derived_ex = init()

In [57]:
# Rule 1
mapping = {"bicep":0.25, "chest": 0.55, "core": 0.20, "leg": 0.45, "back": 0.55, "shoulder": 0.45, "tricep": 0.37}
for key, value in mapping.items():
    df_ex.apply(lambda row: RecommenderLayer.update_weights_muscle_group_matching(df_derived_ex, row, key, value), axis = 1)

In [58]:
df_derived_ex.head(10)

,id,name,exercise_importance,exercise_cost,exercise_recurrance_factor_per_cycle,exercise_rpe,exercise_overall_fun_factor,exercise_exhaustion_factor,missed_in_cycle_count
0,1,bicep_curl,1.25,NaN,NaN,NaN,NaN,NaN,NaN
1,2,burpee,2.20,NaN,NaN,NaN,NaN,NaN,NaN
2,3,deadlift,2.00,NaN,NaN,NaN,NaN,NaN,NaN
3,4,squat,1.45,NaN,NaN,NaN,NaN,NaN,NaN
4,5,bench_press,2.37,NaN,NaN,NaN,NaN,NaN,NaN
5,6,push_up,2.57,NaN,NaN,NaN,NaN,NaN,NaN
6,7,pull_up,1.80,NaN,NaN,NaN,NaN,NaN,NaN
7,8,lat_pulldown,1.80,NaN,NaN,NaN,NaN,NaN,NaN
8,9,leg_press,1.45,NaN,NaN,NaN,NaN,NaN,NaN
9,10,leg_extension,1.45,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
df_ex, df_workout, df_user, df_derived_ex = init()

In [60]:
# Rule 2:
'''
If Age in bracket 18 - 30 -> Improve asthetic muscles -> bicep and tricep and shoulder
'''
import Constants
age_lower = 18
age_higher = 40
age_user = int(df_user[df_user['name'] == Constants.USER_NAME]['age'].item())
if age_lower <= age_user and age_user <= age_higher:
    weight_to_improve = 0.55
    mapping = {"bicep":0.25, "core": 0.25, "shoulder": 0.25, "tricep": 0.25}
    for key, value in mapping.items():
        df_ex.apply(lambda row: RecommenderLayer.update_weights_muscle_group_matching(df_derived_ex, row, key, value), axis = 1)

In [61]:
df_derived_ex.head(10)

,id,name,exercise_importance,exercise_cost,exercise_recurrance_factor_per_cycle,exercise_rpe,exercise_overall_fun_factor,exercise_exhaustion_factor,missed_in_cycle_count
0,1,bicep_curl,1.25,NaN,NaN,NaN,NaN,NaN,NaN
1,2,burpee,1.25,NaN,NaN,NaN,NaN,NaN,NaN
2,3,deadlift,1.00,NaN,NaN,NaN,NaN,NaN,NaN
3,4,squat,1.00,NaN,NaN,NaN,NaN,NaN,NaN
4,5,bench_press,1.50,NaN,NaN,NaN,NaN,NaN,NaN
5,6,push_up,1.75,NaN,NaN,NaN,NaN,NaN,NaN
6,7,pull_up,1.25,NaN,NaN,NaN,NaN,NaN,NaN
7,8,lat_pulldown,1.25,NaN,NaN,NaN,NaN,NaN,NaN
8,9,leg_press,1.00,NaN,NaN,NaN,NaN,NaN,NaN
9,10,leg_extension,1.00,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_ex, df_workout, df_user, df_derived_ex = init()

In [49]:
# Rule 3:
muscle_group_which_user_is_focusing = df_user[df_user['name'] == Constants.USER_NAME]['focused_muscle_group'].item().split()
for i in muscle_group_which_user_is_focusing:
    weight_to_improve = 0.55
    df_ex.apply(lambda row: RecommenderLayer.update_weights_muscle_group_matching(df_derived_ex, row, i, weight_to_improve), axis = 1)

In [50]:
df_derived_ex.head(10)

,id,name,exercise_importance,exercise_cost,exercise_recurrance_factor_per_cycle,exercise_rpe,exercise_overall_fun_factor,exercise_exhaustion_factor,missed_in_cycle_count
0,1,bicep_curl,1.80,NaN,NaN,NaN,NaN,NaN,NaN
1,2,burpee,1.80,NaN,NaN,NaN,NaN,NaN,NaN
2,3,deadlift,2.10,NaN,NaN,NaN,NaN,NaN,NaN
3,4,squat,1.55,NaN,NaN,NaN,NaN,NaN,NaN
4,5,bench_press,2.05,NaN,NaN,NaN,NaN,NaN,NaN
5,6,push_up,2.30,NaN,NaN,NaN,NaN,NaN,NaN
6,7,pull_up,2.35,NaN,NaN,NaN,NaN,NaN,NaN
7,8,lat_pulldown,2.35,NaN,NaN,NaN,NaN,NaN,NaN
8,9,leg_press,1.55,NaN,NaN,NaN,NaN,NaN,NaN
9,10,leg_extension,1.55,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:

df_ex, df_workout, df_user, df_derived_ex = init()

2023-05-11 06:24:07.036172: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-11 06:24:07.038596: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-11 06:24:07.078499: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-11 06:24:07.079295: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-11 06:24:07.792781: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [5]:
if df_user[df_user['name'] == Constants.USER_NAME]['goal'].item() == "strength":
    type_of_musc_to_penalise = "cardio"
    weight_cost = -0.25
    df_ex.apply(lambda row: RecommenderLayer.increase_cost_of_type(df_derived_ex, row, type_of_musc_to_penalise, weight_cost), axis = 1)

True
In
0
-0.25
In
0.0
-0.25
In
0.0
-0.25
In
0.0
-0.25
In
0.0
-0.25
In
0.0
-0.25
In
0.0
-0.25
In
0.0
-0.25
In
0.0
-0.25
In
0.0
-0.25
In
0.0
-0.25


In [6]:
df_derived_ex[df_derived_ex['exercise_cost'] < 0].head(10)

,id,name,exercise_importance,exercise_cost,exercise_recurrance_factor_per_cycle,exercise_rpe,exercise_overall_fun_factor,exercise_exhaustion_factor,missed_in_cycle_count
1,2,burpee,1,-0.25,NaN,NaN,NaN,NaN,NaN
22,23,squat_jump,1,-0.25,NaN,NaN,NaN,NaN,NaN
23,24,jumping_jacks,1,-0.25,NaN,NaN,NaN,NaN,NaN
24,25,high_knees,1,-0.25,NaN,NaN,NaN,NaN,NaN
25,26,jump_rope,1,-0.25,NaN,NaN,NaN,NaN,NaN
26,27,mountain_climbers,1,-0.25,NaN,NaN,NaN,NaN,NaN
27,28,running,1,-0.25,NaN,NaN,NaN,NaN,NaN
28,29,cycling,1,-0.25,NaN,NaN,NaN,NaN,NaN
29,30,swimming,1,-0.25,NaN,NaN,NaN,NaN,NaN
48,49,elliptical,1,-0.25,NaN,NaN,NaN,NaN,NaN


In [2]:

cc = ServiceLayerDao.DaoLayer()
exercise_table = cc.read_table(Constants.TABLE_EXERCISE)
exercise_table.head(10)

,id,name,exercise_type,muscles_targeted,calories_10_min,fun_factor,is_compound,how_critical
0,1,bicep_curl,strength,bicep,40.0,4,0,2
1,2,burpee,cardio,chest core leg,110.0,2,1,4
2,3,deadlift,strength,back leg,85.0,3,1,5
3,4,squat,strength,leg,65.0,4,1,5
4,5,bench_press,strength,chest shoulder tricep,65.0,4,1,5
5,6,push_up,strength,chest core shoulder tricep,80.0,3,1,5
6,7,pull_up,strength,back bicep,65.0,2,1,5
7,8,lat_pulldown,strength,back bicep,50.0,3,1,3
8,9,leg_press,strength,leg,65.0,2,1,4
9,10,leg_extension,strength,leg,40.0,3,0,3


In [4]:
# Rule 8: If exercise is compound, then boost its importance
weight_cost_for_compound = 0.35
df_ex, df_workout, df_user, df_derived_ex = init()
df_ex.apply(lambda row: RecommenderLayer.increase_compound(df_derived_ex, row, weight_cost_for_compound), axis = 1)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
30    None
31    None
32    None
33    None
34    None
35    None
36    None
37    None
38    None
39    None
40    None
41    None
42    None
43    None
44    None
45    None
46    None
47    None
48    None
49    None
50    None
dtype: object

In [4]:
filter_ex = df_ex.nlargest(10, 'calories_10_min')

In [5]:
filter_ex.head(10)

,id,name,exercise_type,muscles_targeted,calories_10_min,fun_factor,is_compound,how_critical
27,28,running,cardio,leg,125.0,4,0,5
34,35,stair_climbing,endurance,core leg,125.0,3,1,4
37,38,box_jumps,endurance,leg,125.0,3,1,4
39,40,tire_flips,strongman,core leg,125.0,4,1,5
40,41,kettlebell_swing,strongman,leg,125.0,4,1,5
29,30,swimming,cardio,back bicep chest leg shoulder tricep core,120.0,5,0,5
1,2,burpee,cardio,chest core leg,110.0,2,1,4
22,23,squat_jump,cardio,leg,100.0,3,1,4
25,26,jump_rope,cardio,back bicep chest leg shoulder tricep core,100.0,4,0,3
28,29,cycling,cardio,leg,100.0,4,0,5


In [7]:
df_derived_ex.head(10)

,id,name,exercise_importance,exercise_cost,exercise_recurrance_factor_per_cycle,exercise_rpe,exercise_overall_fun_factor,exercise_exhaustion_factor,missed_in_cycle_count
0,1,bicep_curl,1,0,NaN,NaN,NaN,NaN,NaN
1,2,burpee,1,0,NaN,NaN,NaN,NaN,NaN
2,3,deadlift,1,0,NaN,NaN,NaN,NaN,NaN
3,4,squat,1,0,NaN,NaN,NaN,NaN,NaN
4,5,bench_press,1,0,NaN,NaN,NaN,NaN,NaN
5,6,push_up,1,0,NaN,NaN,NaN,NaN,NaN
6,7,pull_up,1,0,NaN,NaN,NaN,NaN,NaN
7,8,lat_pulldown,1,0,NaN,NaN,NaN,NaN,NaN
8,9,leg_press,1,0,NaN,NaN,NaN,NaN,NaN
9,10,leg_extension,1,0,NaN,NaN,NaN,NaN,NaN


In [6]:

df_ex, df_workout, df_user, df_derived_ex = init()

weight_penalty = -0.32
# Top 10 burnout need to be penalised
cals_burnout_limit = 10

filter_ex.apply(lambda row: RecommenderLayer.penalty_burnout(df_ex, row, weight_penalty), axis = 1)

KeyError: 'exercise_cost'

In [9]:
df_derived_ex[df_derived_ex['id'] == 28]['exercise_cost'].item()

0

In [5]:
df_derived_ex.head(10)

,id,name,exercise_importance,exercise_cost,exercise_recurrance_factor_per_cycle,exercise_rpe,exercise_overall_fun_factor,exercise_exhaustion_factor,missed_in_cycle_count
0,1,bicep_curl,1,0.00,NaN,NaN,NaN,NaN,NaN
1,2,burpee,1,0.35,NaN,NaN,NaN,NaN,NaN
2,3,deadlift,1,0.35,NaN,NaN,NaN,NaN,NaN
3,4,squat,1,0.35,NaN,NaN,NaN,NaN,NaN
4,5,bench_press,1,0.35,NaN,NaN,NaN,NaN,NaN
5,6,push_up,1,0.35,NaN,NaN,NaN,NaN,NaN
6,7,pull_up,1,0.35,NaN,NaN,NaN,NaN,NaN
7,8,lat_pulldown,1,0.35,NaN,NaN,NaN,NaN,NaN
8,9,leg_press,1,0.35,NaN,NaN,NaN,NaN,NaN
9,10,leg_extension,1,0.00,NaN,NaN,NaN,NaN,NaN


In [53]:
columns_to_replace_nan_to_zero = ["exercise_importance", "exercise_cost", "exercise_recurrance_factor_per_cycle", "exercise_rpe", "exercise_overall_fun_factor", "exercise_exhaustion_factor", "missed_in_cycle_count"]
for i in columns_to_replace_nan_to_zero:
    df_derived_ex[i].fillna(0, inplace=True)

In [54]:
df_derived_ex.head(10)

,id,name,exercise_importance,exercise_cost,exercise_recurrance_factor_per_cycle,exercise_rpe,exercise_overall_fun_factor,exercise_exhaustion_factor,missed_in_cycle_count
0,1,bicep_curl,1,0.0,0.0,0.0,0.0,0.0,0.0
1,2,burpee,1,0.0,0.0,0.0,0.0,0.0,0.0
2,3,deadlift,1,0.0,0.0,0.0,0.0,0.0,0.0
3,4,squat,1,0.0,0.0,0.0,0.0,0.0,0.0
4,5,bench_press,1,0.0,0.0,0.0,0.0,0.0,0.0
5,6,push_up,1,0.0,0.0,0.0,0.0,0.0,0.0
6,7,pull_up,1,0.0,0.0,0.0,0.0,0.0,0.0
7,8,lat_pulldown,1,0.0,0.0,0.0,0.0,0.0,0.0
8,9,leg_press,1,0.0,0.0,0.0,0.0,0.0,0.0
9,10,leg_extension,1,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Dump
df_ex.apply(lambda row: update_weights(row), axis=1)
# Join the datasets
merged_df = pd.merge(df_ex, df_derived_ex, on=['id', 'name'])
